## Test Cases for Auxiliary Statements used by Apache Spark
**Spark SQL 3.3 Auxiliary Statements** 
https://spark.apache.org/docs/3.3.0/sql-ref-syntax.html#auxiliary-statements

**<mark>Excludes</mark>:**


To store these results configure data <mark>**storage account and container**</mark>.

In [ ]:
!pip install unittest-xml-reporting xmltodict

## Configure Result Storage Location

In [ ]:
storage_account=""
result_container=""
# change this to accessible URL
add_jar_url = "https://github.com/sethiaarun/azureSynapseSparkUnitTest/releases/download/1.0/businessfunction-1.0.jar"
add_file_url = "https://github.com/apache/spark/blob/master/examples/src/main/python/wordcount.py"
archive_file_url = "https://resources.lendingclub.com/LoanStats3a.csv.zip"

## Initialize Common Variables for the test run

In [ ]:
import time

# Don't change these variables
TEST_SUITE= "SPARK_SQL_AUX_STATEMENTS"
RESULT_FILE_NAME="aux_stmt_test_result.parquet"
RAW_RESULT_FILE_NAME="raw_aux_stmt_test_result.parquet"
# Test Run ID
TEST_RUN_ID= round(time.time()*1000)
# Test platform
PLATFORM = "nameoftheplatform"
# Prefix for all tables
PREFIX = PLATFORM
SUFFIX = TEST_RUN_ID
# Spark SQL function
sql=spark.sql

### Set Common Spark Configurations

In [ ]:
sql("set hive.exec.dynamic.partition.mode=nonstrict")

### Table Auxiliary Statements - DELTA FORMAT

In [ ]:
import unittest
from pyspark.sql.utils import AnalysisException

class AuxDeltaTableStmtTest(unittest.TestCase):
    """Auxiliary Statements test cases - using delta table"""
    
    table_name = f"{PREFIX}_student_aux_stmt_delta_table_{SUFFIX}"
    view_name = f"{PREFIX}_student_aux_stmt_delta_table_view_{SUFFIX}"      
   
    @classmethod
    def setUpClass(cls):
        sql_cmd = f"CREATE TABLE {cls.table_name} (id INT, name STRING) USING delta PARTITIONED BY (age INT) \
                          TBLPROPERTIES ('created.by.user' = 'Testcases')"
        try:
            sql(sql_cmd)
            sql(f"INSERT INTO {cls.table_name} VALUES \
                 (1,'a',10),(2,'b',20),(3,'c',30);")
        except Exception as ex:
            msg={'command':'AlterTable:Setup failed','status':'fail'}
            cls.fail(f"{msg}")

    
    def test_delta_aux_stmt_001_show_column(self):
        """Show Columns"""
        sql_cmd = f"SHOW COLUMNS IN {self.table_name};"
        try:
            row_count=sql(sql_cmd).count()
            self.assertEqual(row_count,3)
        except Exception as ex:
            msg={'command':'SHOW COLUMNS for Delta Table','status':'fail'}
            self.fail(f"{msg}")
    
    def test_delta_aux_stmt_002_show_create_table_stmt(self):
        """Show Create Table Statement"""
        sql_cmd = f"SHOW CREATE TABLE {self.table_name} AS SERDE;"
        try:
            row_count=sql(sql_cmd).select("createtab_stmt").count()
            self.assertEqual(row_count,1)
        except Exception as ex:
            msg={'command':'SHOW CREATE TABLE Statement for Delta Table','status':'fail'}
            self.fail(f"{msg}")

    def test_delta_aux_stmt_003_show_partition_table_stmt(self):
        """Show Partition Table Statement"""
        sql_cmd = f"SHOW PARTITIONS {self.table_name};"
        try:
            row_count=sql(sql_cmd)
            self.assertEqual(row_count.select("partition").count(),3)
            row_count=sql(f"SHOW PARTITIONS {self.table_name} PARTITION (age = '10');").select("partition").count()
            self.assertEqual(row_count,1)
        except Exception as ex:
            msg={'command':'SHOW PARTITIONS Table Statement for Delta Table','status':'fail'}
            self.fail(f"{msg}")

    def test_delta_aux_stmt_004_show_table_extended_stmt(self):
        """Show Table Extended Table Statement"""
        sql_cmd = f"SHOW TABLE EXTENDED  LIKE '{self.table_name}';"
        try:
            row_count=sql(sql_cmd).select("tableName").count()
            self.assertEqual(row_count,1)
        except Exception as ex:
            msg={'command':'SHOW TABLE EXTENDED LIKE  Statement for Delta Table','status':'fail'}
            self.fail(f"{msg}")

    def test_delta_aux_stmt_005_show_table_extended_partition_stmt(self):
        """Show Table Extended Table Statement with Partition"""
        sql_cmd = f"SHOW TABLE EXTENDED  LIKE '{self.table_name}' PARTITION (age=10);"
        try:
            row_count=sql(sql_cmd).select("tableName").count()
            self.assertEqual(row_count,1)
        except Exception as ex:
            msg={'command':'SHOW TABLE EXTENDED LIKE PARTITION for Delta Table','status':'fail'}
            self.fail(f"{msg}")

    def test_delta_aux_stmt_006_show_tables_stmt(self):
        """Show Table Statement"""
        sql_cmd1 = f"SHOW TABLES;"
        sql_cmd2 = f"SHOW TABLES LIKE 'student*|test*';"
        try:
            row_count=sql(sql_cmd1).select("tableName").count()
            self.assertGreaterEqual(row_count,1)
            row_count=sql(sql_cmd2).select("tableName").count()
            self.assertGreaterEqual(row_count,1)
        except Exception as ex:
            msg={'command':'SHOW TABLES and SHOW TABLES LIKE for Delta Table','status':'fail'}
            self.fail(f"{msg}")

    def test_delta_aux_stmt_007_show_tables_prop_stmt(self):
        """Show Table Statement"""
        sql_cmd1 = f"SHOW TBLPROPERTIES {self.table_name};"
        sql_cmd2 = f"SHOW TBLPROPERTIES {self.table_name} (created.by.user);"
        try:
            row_count=sql(sql_cmd1).select("key").count()
            self.assertGreaterEqual(row_count,1)
            row_count=sql(sql_cmd2).select("value").count()
            self.assertGreaterEqual(row_count,1)
        except Exception as ex:
            msg={'command':'SHOW TBLPROPERTIES for Delta Table','status':'fail'}
            self.fail(f"{msg}")
    
    def test_delta_aux_stmt_008_create_view(self):
        """create view"""
        sql_cmd = f"CREATE OR REPLACE VIEW  {self.view_name} (id COMMENT 'ID', name COMMENT 'Name') \
                    COMMENT 'View for student table' \
                    AS SELECT ID,name FROM {self.table_name};"
        try:
            sql(sql_cmd)
        except Exception as ex:
            msg={'command':'CREATE OR REPLACE VIEW for Delta Table','status':'fail'}
            self.fail(f"{msg}")
    
    def test_delta_aux_stmt_009_show_view(self):
        """show view"""
        try:
            view_exist=sql(f"SHOW VIEWS").filter(f"viewName='{self.view_name}'").count()
            self.assertEqual(view_exist,1)
        except Exception as ex:
            msg={'command':'SHOW VIEWS for Delta Table','status':'fail'}
            self.fail(f"{msg}")

    
    def test_delta_aux_stmt_010_cache_table_stmt(self):
        """Cache and UnCache Table"""
        cache_table_name = f"{PREFIX}_student_aux_stmt_delta_table_cache_{SUFFIX}"
        sql_cmd = f"CACHE TABLE {cache_table_name}  OPTIONS ('storageLevel' 'DISK_ONLY') SELECT * FROM {self.table_name}"
        uncache_sql_cmd = f"UNCACHE TABLE {cache_table_name}"
        try:
            sql(sql_cmd)
            row_count=sql(f"SELECT * FROM {cache_table_name}").count()
            self.assertGreaterEqual(row_count,1)
            sql(uncache_sql_cmd)
        except Exception as ex:
            msg={'command':'CACHE TABLE and UNCACHE TABLE for Delta Table','status':'fail'}
            self.fail(f"{msg}")

        finally:
            sql(f"DROP TABLE IF EXISTS {cache_table_name}")

    def test_delta_aux_stmt_011_stat_noscan(self):
        """Delta Table  Analyze Table Compute Statistics NoScan"""
        sql_cmd = f"ANALYZE TABLE {self.table_name} COMPUTE STATISTICS NOSCAN;"
        try:
            sql(sql_cmd)
            row_count=sql(f"DESC EXTENDED {self.table_name}").where("col_name='Statistics'").count()
            self.assertEqual(row_count,1)
        except Exception as ex:
            msg={'command':'ANALYZE TABLE COMPUTE STATISTICS NOSCAN and DESC EXTENDED for Delta Table','status':'fail'}
            self.fail(f"{msg}")
    
    def test_delta_aux_stmt_012_describe_query(self):
        """Describe query"""
        sql_cmd = f"DESCRIBE QUERY SELECT age, sum(age) FROM {self.table_name} GROUP BY age;"
        try:
            row_count=sql(sql_cmd).select("col_name").count()
            self.assertEqual(row_count,2)
        except Exception as ex:
            msg={'command':'DESCRIBE QUERY for Delta format','status':'fail'}
            self.fail(f"{msg}")

    def test_delta_aux_stmt_012_describe_table(self):
        """Describe table"""
        sql_cmd = f"DESCRIBE TABLE {self.table_name}"
        try:
            row_count=sql(sql_cmd).select("col_name").count()
            self.assertGreaterEqual(row_count,1)
        except Exception as ex:
            msg={'command':'DESCRIBE QUERY for Delta format','status':'fail'}
            self.fail(f"{msg}")


    @classmethod
    def tearDownClass(cls):
        """tear down"""
        try:
            sql(f"DROP TABLE IF EXISTS {cls.table_name}")
        except Exception as e:
            print(f"table drop failed: {cls.table_name}")

        try:
            sql(f"DROP VIEW IF EXISTS {cls.view_name}")
        except Exception as e:
            print(f"View drop failed: {cls.view_name}")

### Table Auxiliary Statements - DEFAULT FORMAT

In [ ]:
import unittest
from pyspark.sql.utils import AnalysisException

class AuxTableStmtTest(unittest.TestCase):
    """Auxiliary Statements test cases - using default format"""
    
    table_name = f"{PREFIX}_student_aux_stmt_default_table_{SUFFIX}"
    view_name = f"{PREFIX}_student_aux_stmt_default_table_view_{SUFFIX}"
   
    @classmethod
    def setUpClass(cls):
        sql_cmd = f"CREATE TABLE {cls.table_name} (id INT, name STRING) PARTITIONED BY (age INT) \
                          TBLPROPERTIES ('created.by.user' = 'Testcases')"
        try:
            sql(sql_cmd)
            sql(f"INSERT INTO {cls.table_name} VALUES \
                 (1,'a',10),(2,'b',20),(3,'c',30);")
        except Exception as ex:
            msg={'command':'AlterTable:Setup failed','status':'fail'}
            cls.fail(f"{msg}")

    
    def test_default_aux_stmt_001_show_column(self):
        """Show Columns"""
        sql_cmd = f"SHOW COLUMNS IN {self.table_name};"
        try:
            row_count=sql(sql_cmd).count()
            self.assertEqual(row_count,3)
        except Exception as ex:
            msg={'command':'SHOW COLUMNS for Default format','status':'fail'}
            self.fail(f"{msg}")
    
    def test_default_aux_stmt_002_show_create_table_stmt(self):
        """Show Create Table Statement"""
        sql_cmd = f"SHOW CREATE TABLE {self.table_name} AS SERDE;"
        try:
            row_count=sql(sql_cmd).select("createtab_stmt").count()
            self.assertEqual(row_count,1)
        except Exception as ex:
            msg={'command':'SHOW CREATE TABLE Statement for Default format','status':'fail'}
            self.fail(f"{msg}")

    def test_default_aux_stmt_003_show_partition_table_stmt(self):
        """Show Partition Table Statement"""
        sql_cmd = f"SHOW PARTITIONS {self.table_name};"
        try:
            row_count=sql(sql_cmd)
            self.assertEqual(row_count.select("partition").count(),3)
            row_count=sql(f"SHOW PARTITIONS {self.table_name} PARTITION (age = '10');").select("partition").count()
            self.assertEqual(row_count,1)
        except Exception as ex:
            msg={'command':'SHOW PARTITIONS Table Statement for Default format','status':'fail'}
            self.fail(f"{msg}")

    def test_default_aux_stmt_004_show_table_extended_stmt(self):
        """Show Table Extended Table Statement"""
        sql_cmd = f"SHOW TABLE EXTENDED  LIKE '{self.table_name}';"
        try:
            row_count=sql(sql_cmd).select("tableName").count()
            self.assertEqual(row_count,1)
        except Exception as ex:
            msg={'command':'SHOW TABLE EXTENDED LIKE  Statement for Delta Table','status':'fail'}
            self.fail(f"{msg}")

    def test_default_aux_stmt_005_show_table_extended_partition_stmt(self):
        """Show Table Extended Table Statement with Partition"""
        sql_cmd = f"SHOW TABLE EXTENDED  LIKE '{self.table_name}' PARTITION (age=10);"
        try:
            row_count=sql(sql_cmd).select("tableName").count()
            self.assertEqual(row_count,1)
        except Exception as ex:
            msg={'command':'SHOW TABLE EXTENDED LIKE PARTITION for Default format','status':'fail'}
            self.fail(f"{msg}")

    def test_default_aux_stmt_006_show_tables_stmt(self):
        """Show Table Statement"""
        sql_cmd1 = f"SHOW TABLES;"
        sql_cmd2 = f"SHOW TABLES LIKE 'student*|test*';"
        try:
            row_count=sql(sql_cmd1).select("tableName").count()
            self.assertGreaterEqual(row_count,1)
            row_count=sql(sql_cmd2).select("tableName").count()
            self.assertGreaterEqual(row_count,1)
        except Exception as ex:
            msg={'command':'SHOW TABLES and SHOW TABLES LIKE for Default format','status':'fail'}
            self.fail(f"{msg}")

    def test_default_aux_stmt_007_show_tables_prop_stmt(self):
        """Show Table Statement"""
        sql_cmd1 = f"SHOW TBLPROPERTIES {self.table_name};"
        sql_cmd2 = f"SHOW TBLPROPERTIES {self.table_name} (created.by.user);"
        try:
            row_count=sql(sql_cmd1).select("key").count()
            self.assertGreaterEqual(row_count,1)
            row_count=sql(sql_cmd2).select("value").count()
            self.assertGreaterEqual(row_count,1)
        except Exception as ex:
            msg={'command':'SHOW TBLPROPERTIES for Default format','status':'fail'}
            self.fail(f"{msg}")
    
    def test_default_aux_stmt_008_create_view(self):
        """create view"""
        sql_cmd = f"CREATE OR REPLACE VIEW  {self.view_name} (id COMMENT 'ID', name COMMENT 'Name') \
                    COMMENT 'View for student table' \
                    AS SELECT ID,name FROM {self.table_name};"
        try:
            sql(sql_cmd)
        except Exception as ex:
            msg={'command':'CREATE OR REPLACE VIEW for Default format','status':'fail'}
            self.fail(f"{msg}")
    
    def test_default_aux_stmt_009_show_view(self):
        """show view"""
        try:
            view_exist=sql(f"SHOW VIEWS").filter(f"viewName='{self.view_name}'").count()
            self.assertEqual(view_exist,1)
        except Exception as ex:
            msg={'command':'SHOW VIEWS for Default format','status':'fail'}
            self.fail(f"{msg}")

    
    def test_default_aux_stmt_010_cache_table_stmt(self):
        """Cache and UnCache Table"""
        sql_cmd = f"CACHE TABLE {self.cache_table_name}  OPTIONS ('storageLevel' 'DISK_ONLY') SELECT * FROM {self.table_name}"
        uncache_sql_cmd = f"UNCACHE TABLE {self.cache_table_name}"
        try:
            sql(sql_cmd)
            row_count=sql(f"SELECT * FROM {self.cache_table_name}").count()
            self.assertGreaterEqual(row_count,1)
            sql(uncache_sql_cmd)
        except Exception as ex:
            msg={'command':'CACHE TABLE and UNCACHE TABLE for Default format','status':'fail'}
            self.fail(f"{msg}")

    def test_default_aux_stmt_011_stat_noscan(self):
        """Analyze Table Compute Statistics NoScan"""
        sql_cmd = f"ANALYZE TABLE {self.table_name} COMPUTE STATISTICS NOSCAN;"
        try:
            sql(sql_cmd)
            row_count=sql(f"DESC EXTENDED {self.table_name}").where("col_name='Statistics'").count()
            self.assertEqual(row_count,1)
        except Exception as ex:
            msg={'command':'ANALYZE TABLE COMPUTE STATISTICS NOSCAN and DESC EXTENDED for Default format','status':'fail'}
            self.fail(f"{msg}")

    def test_default_aux_stmt_012_describe_query(self):
        """Describe query"""
        sql_cmd = f"DESCRIBE QUERY SELECT age, sum(age) FROM {self.table_name} GROUP BY age;"
        try:
            row_count=sql(sql_cmd).select("col_name").count()
            self.assertEqual(row_count,2)
        except Exception as ex:
            msg={'command':'DESCRIBE QUERY for Default format','status':'fail'}
            self.fail(f"{msg}")

    @classmethod
    def tearDownClass(cls):
        """tear down"""
        try:
            sql(f"DROP TABLE IF EXISTS {cls.table_name}")
        except Exception as e:
            print(f"table drop failed: {cls.table_name}")

        try:
            sql(f"DROP TABLE IF EXISTS {cls.cache_table_name}")
        except Exception as e:
            print(f"table drop failed: {cls.cache_table_name}")
        
        try:
            sql(f"DROP VIEW  IF EXISTS {cls.view_name}")
        except Exception as e:
            print(f"View drop failed: {cls.view_name}")

### Analyze Table Statements

**Analyze Tables is <mark>not supported for V2 tables.**

In [ ]:
import unittest
from pyspark.sql.utils import AnalysisException

class AnalyzeTableStmtTest(unittest.TestCase):
    """Analyze Table Statements test cases for Non Delta Tables"""
    
    table_name = f"{PREFIX}_student_analyze_table_{SUFFIX}"   
   
    @classmethod
    def setUpClass(cls):
        sql_cmd = f"CREATE TABLE {cls.table_name} (id INT, name STRING) PARTITIONED BY (age INT) \
                          TBLPROPERTIES ('created.by.user' = 'Testcases')"
        try:
            sql(sql_cmd)
            sql(f"INSERT INTO {cls.table_name} VALUES \
                 (1,'a',10),(2,'b',20),(3,'c',30);")
        except Exception as ex:
            msg={'command':'AnalyzeTable:Setup failed','status':'fail'}
            cls.fail(f"{msg}")

    
    def test_analyze_table_stmt_001_stat_partition(self):
        """Analyze Table Compute Statistics with partition"""
        sql_cmd = f"ANALYZE TABLE {self.table_name} PARTITION (age=10) COMPUTE STATISTICS;"
        try:
            sql(sql_cmd)
            row_count=sql(f"DESC EXTENDED {self.table_name} PARTITION (age=10)").where("col_name='Partition Statistics'").count()
            self.assertEqual(row_count,1)
        except Exception as ex:
            msg={'command':'ANALYZE TABLE PARTITION COMPUTE STATISTICS for default format','status':'fail'}
            self.fail(f"{msg}")
    
    def test_analyze_table_stmt_002_stat_for_columns(self):
        """Analyze Table Compute Statistics for Columns"""
        sql_cmd = f"ANALYZE TABLE {self.table_name} COMPUTE STATISTICS FOR COLUMNS name"
        try:
            sql(sql_cmd)
            row_count=sql(f"DESC EXTENDED {self.table_name} name").where("info_name='col_name'").count()
            self.assertEqual(row_count,1)
        except Exception as ex:
            msg={'command':'ANALYZE TABLE COMPUTE STATISTICS FOR COLUMNS for default format','status':'fail'}
            self.fail(f"{msg}")
    

    @classmethod
    def tearDownClass(cls):
        """tear down"""
        try:
            sql(f"DROP TABLE IF EXISTS {cls.table_name}")
        except Exception as e:
            print(f"table drop failed: {cls.table_name}")

### Miscellaneous Auxiliary Statements

In [ ]:
import unittest
from pyspark.sql.utils import AnalysisException

class AuxMiscStmtTest(unittest.TestCase):
    """Auxiliary miscellaneous Statements"""
    
   
    def test_aux_stmt_001_show_schema(self):
        """Show Schema"""
        sql_cmd = f"SHOW SCHEMAS;"
        try:
            row_count=sql(sql_cmd).count()
            self.assertGreaterEqual(row_count,0)
        except Exception as ex:
            msg={'command':'SHOW SCHEMAS','status':'fail'}
            self.fail(f"{msg}")
    
    def test_aux_stmt_001_show_databases(self):
        """Show Databases"""
        sql_cmd = f"SHOW DATABASES;"
        try:
            row_count=sql(sql_cmd).count()
            self.assertGreaterEqual(row_count,1)
        except Exception as ex:
            msg={'command':'SHOW DATABASES','status':'fail'}
            self.fail(f"{msg}")   

    def test_aux_stmt_002_desc_databases(self):
        """describe Databases"""
        sql_cmd = f"SHOW DATABASES;"
        msg={'command':'DESC DATABASE','status':'fail'}
        try:
            databases=sql(sql_cmd).select('namespace').rdd.flatMap(lambda x: x).collect()
            if len(databases)>0:
                row_count=sql(f"DESC DATABASE {databases[0]}").where(f"info_value='{databases[0]}'").count()
                self.assertEqual(row_count,1)
            else:
                self.fail(f"{msg}")  
        except Exception as ex:
            self.fail(f"{msg}")   

    #sql("SHOW DATABASES").select('namespace').rdd.flatMap(lambda x: x).collect()
    def test_aux_stmt_003_show_functions(self):
        """Show Functions"""
        sql_cmd = f"SHOW FUNCTIONS explode;"
        try:
            row_count=sql(sql_cmd).count()
            self.assertEqual(row_count,1)
            row_count=sql("SHOW FUNCTIONS LIKE 't*'").count()
            self.assertGreaterEqual(row_count,1)
            row_count = sql("SHOW FUNCTIONS LIKE 'yea*|windo*'").count()
            self.assertGreaterEqual(row_count,1)
        except Exception as ex:
            msg={'command':'SHOW FUNCTIONS and SHOW FUNCTIONS LIKE','status':'fail'}
            self.fail(f"{msg}")   

    def test_aux_stmt_004_desc_functions(self):
        """describe functions"""
        sql_cmd = f"DESC FUNCTION EXTENDED abs"
        try:
            row_count=sql(sql_cmd).select('function_desc').count()
            self.assertGreaterEqual(row_count,1)
        except Exception as ex:
            msg={'command':'DESC FUNCTION','status':'fail'}
            self.fail(f"{msg}")   

    def test_aux_stmt_005_add_list_jar(self):
        """Add and List Jar Statement"""
        sql_cmd1 = f"ADD JAR {add_jar_url}"
        sql_cmd2= f"LIST JAR {add_jar_url}"
        try:
            sql(sql_cmd1)
            row_count=sql(sql_cmd2).count()
            self.assertGreaterEqual(row_count,1)
        except Exception as ex:
            msg={'command':'ADD JAR and LIST JAR','status':'fail'}
            self.fail(f"{msg}")   

    def test_aux_stmt_006_add_file(self):
        """Add and List File Statement"""
        sql_cmd1 = f"ADD FILE {add_file_url}"
        sql_cmd2= f"LIST FILE {add_file_url}"
        try:
            sql(sql_cmd1)
            row_count=sql(sql_cmd2).count()
            self.assertGreaterEqual(row_count,1)
        except Exception as ex:
            msg={'command':'ADD FILE and LIST FILE','status':'fail'}
            self.fail(f"{msg}")   

    def test_aux_stmt_007_add_archive_file(self):
        """Add and List File Statement"""
        sql_cmd1 = f"ADD ARCHIVE {archive_file_url}"
        sql_cmd2= f"LIST ARCHIVE {archive_file_url}"
        try:
            sql(sql_cmd1)
            row_count=sql(sql_cmd2).count()
            self.assertGreaterEqual(row_count,1)
        except Exception as ex:
            msg={'command':'ADD ARCHIVE and LIST ARCHIVE','status':'fail'}
            self.fail(f"{msg}")   


### Execute Test Cases

In [ ]:
import io
import xmlrunner
loader = unittest.TestLoader()
suite  = unittest.TestSuite()

# add tests to the test suite
suite.addTests(loader.loadTestsFromTestCase(AnalyzeTableStmtTest))
suite.addTests(loader.loadTestsFromTestCase(AuxDeltaTableStmtTest))
suite.addTest(loader.loadTestsFromTestCase(AuxMiscStmtTest))
suite.addTest(loader.loadTestsFromTestCase(AnalyzeTableStmtTest))

# initialize a runner, pass it your suite and run it
out = io.BytesIO()
runner = xmlrunner.XMLTestRunner(output=out)
result = runner.run(suite)

## Report for Test

In [ ]:
from pyspark.sql.functions import col, explode,isnull,from_json, expr, to_json, coalesce, lit
from pyspark.sql.types import StructType,StructField,StringType
import json
import xmltodict

dict_result=xmltodict.parse(out.getvalue())
json_result = json.loads(json.dumps(dict_result,indent=4).replace('@',''))
test_suites=json_result['testsuites']['testsuite']

df = spark.read.json(sc.parallelize([test_suites]))
fail_schema = StructType([
  StructField("command", StringType(), True),
  StructField("status", StringType(),  True)
])

test_cases_df= df.withColumn('ts',explode('testcase')).drop(col('testcase'))

if "failure:" in test_cases_df.schema.simpleString():
 explode_df= test_cases_df.withColumn('fail',from_json(col('ts.failure.message'),fail_schema)).drop(col('ts.failure'))
else:
 explode_df= test_cases_df.withColumn("fail",from_json(expr("to_json(named_struct('command', '', 'status', 'pass'))"),fail_schema))
 
df_test_result=explode_df.select(col("errors").alias("errorInSuite"),col("failures").alias("failedInSuite"),col("name").alias("suitename"),\
      "skipped",col("tests").alias("totalTest"), col("timestamp").alias("executionTime"),col("ts.name").alias("testCaseName"), \
       col("ts.time").alias("testCaseTime"),coalesce(col("fail.command"), lit("")).alias("failcommand"),coalesce(col("fail.status"), lit("pass")).alias("status"))

df_test_result_summary=df_test_result.select(col('errorInSuite'),col('failedInSuite'),col('suitename'),col('skipped'),col('totaltest')).distinct()

if (len(storage_account)>0 and len(result_container)>0):
    # save result to storage
    storage_path = f"abfs://{result_container}@{storage_account}.dfs.core.windows.net/{TEST_RUN_ID}/{PLATFORM}/{TEST_SUITE}"
    # write raw results
    df.write.parquet(f"{storage_path}/{RAW_RESULT_FILE_NAME}")
    # write transformed results
    df_test_result.write.parquet(f"{storage_path}/{RESULT_FILE_NAME}") 
    # write transformed results
    df_test_result_summary.write.parquet(f"{storage_path}/{RESULT_FILE_NAME}_summary") 
else:
    print("configure storage path to store results")
    df_test_result.show(200,False)    
    df_test_result_summary.show(100,False)